In [1]:
import pymysql
import numpy as np
import pandas as pd
import xmltodict
from pandas import json_normalize
import urllib.request
import requests
import base64
import requests
from bs4 import BeautifulSoup
import re
import json
from multiprocessing import Pool
from datetime import datetime
from functools import reduce
import time

In [43]:
# read AL postcodes
AL = pd.read_csv('AL postcodes.csv')
AL['Postcode'] = AL['Postcode'].str.replace(" ","").str.lower()
AL.head()

,Postcode,In Use?,Latitude,Longitude,Easting,Northing,Grid Ref,County,District,Ward,...,Postcode area,Postcode district,Police force,Water company,Plus Code,Average Income,Sewage Company,Travel To Work Area,ITL level 2,ITL level 3
0,al11aa,No,51.749084,-0.341337,514600.0,206900.0,TL146068,Hertfordshire,St Albans,Verulam,...,AL,AL1,Hertfordshire,NaN,9C3XPMX5+JF,68900.0,NaN,Luton,Bedfordshire and Hertfordshire,Hertfordshire CC
1,al11ab,No,51.747828,-0.301865,517328.0,206823.0,TL173068,Hertfordshire,St Albans,Ashley,...,AL,AL1,Hertfordshire,NaN,9C3XPMXX+47,66500.0,NaN,Luton,Bedfordshire and Hertfordshire,Hertfordshire CC
2,al11ad,No,51.747828,-0.301865,517328.0,206823.0,TL173068,Hertfordshire,St Albans,Ashley,...,AL,AL1,Hertfordshire,NaN,9C3XPMXX+47,66500.0,NaN,Luton,Bedfordshire and Hertfordshire,Hertfordshire CC
3,al11ae,No,51.747828,-0.301865,517328.0,206823.0,TL173068,Hertfordshire,St Albans,Ashley,...,AL,AL1,Hertfordshire,NaN,9C3XPMXX+47,66500.0,NaN,Luton,Bedfordshire and Hertfordshire,Hertfordshire CC
4,al11af,No,51.749084,-0.341337,514600.0,206900.0,TL146068,Hertfordshire,St Albans,Verulam,...,AL,AL1,Hertfordshire,NaN,9C3XPMX5+JF,68900.0,NaN,Luton,Bedfordshire and Hertfordshire,Hertfordshire CC


In [4]:
# Connect to the database
db = pymysql.connect(host='xxxx',
                     user='xxxx', 
                     password = 'xxxx')

cursor = db.cursor()

In [6]:
# Select version
cursor.execute("select version()")

1

In [7]:
#Fetch a row from the query
data = cursor.fetchone()

In [8]:
#Show data
data

('8.0.23',)

In [44]:
# Drop database table
#sql = '''drop database postcodes'''
#cursor.execute(sql)

In [13]:
# create table
try:
    conn = pymysql.connect(host='xxxx', 
                           user='xxxx', 
                           password='xxxx')

    cursor = conn.cursor()
    cursor.execute("select database();")
    record = cursor.fetchone()
    print("You're connected to database: ", record)
    print('Creating table....')
    
    sql = '''create database postcodes'''
    cursor.execute(sql)
    cursor.connection.commit()
    sql = '''use postcodes'''
    cursor.execute(sql)

    cursor.execute('''CREATE TABLE properties (
                    address VARCHAR(255), 
                    propertyType VARCHAR(255),
                    bedrooms INT(2), 
                    bathrooms INT(2), 
                    transactions_price VARCHAR(255), 
                    transactions_date VARCHAR(255), 
                    transactions_tenure VARCHAR(255), 
                    lat VARCHAR(255), 
                    lgt VARCHAR(255), 
                    detailUrl VARCHAR(255), 
                    transactions_date_dt DATE, 
                    postcode VARCHAR(255)
                    )''')
    
    print("addresses table is created....")

except Error as e:
    print("Error while connecting to MySQL", e)

You're connected to database:  (None,)
Creating table....
addresses table is created....


In [15]:
def df_drop_df(df_1, df_2, l_columns):
    """
    Eliminates all equal rows present in dataframe 1 (df_1) from dataframe 2 (df_2) depending on a subset of columns (l_columns)

    :param df_1: dataframe that defines which rows to be removed
    :param df_2: dataframe that is reduced
    :param l_columns: list of column names, present in df_1 and df_2, that is used for the comparison

    :return df_out: final dataframe
    """
    df_1r = df_1[l_columns]
    df_2r = df_2[l_columns].reset_index()

    df_m = pd.merge(df_1r, df_2r, on=l_columns, how='inner')
    row_indexes_m = df_m['index'].to_list()

    row_indexes_df_2 = df_2.index.to_list()
    row_indexes_out = [x for x in row_indexes_df_2 if x not in row_indexes_m]

    df_out = df_2.loc[row_indexes_out]
    return df_out


def get_data(user_input, user_input_year):
    """
    Scrap the data from Rightmove using user inputs on postcode and year. Choosing specific year will apply a filter 
    to fetch only data from the beginning of that year until now.
    
    :param user_input: postcode (full or partial)
    :param user_input_year: year

    :return data_year: data frame containing information on past sales in that postcode
    """    
    
    def urls(postcode):
        """
        Return list of urls (Rightmove has results over 40 pages per postcode)
        
        :param postcode: user input on postcode
    
        :return url_list: list of 40 pages for scraping
        """          
        url_list = list()
        count = 1
        for i in range(1,41):
            url1 = 'https://www.rightmove.co.uk/house-prices/'
            url2 = '.html?page='
            a = url1 + postcode + url2 + str(count)
            url_list.append(a)
            count+=1
            
        return url_list
    
    post_list_rightmove = urls(user_input)
    
    def get_data_postcode(post_list):   
        """
        Scrap data for user postcode for each one of the Rightmove 40 pages
        
        :param post_list: list of 40 pages for scraping
    
        :return data: concatenation of property sales data from all 40 pages
        """   
        
        address = list()
        propertyType = list()
        bedrooms = list()
        bathrooms = list()
        transactions_price = list()
        transactions_date = list()
        transactions_tenure = list()
        lat = list()
        lgt = list()
        detailUrl = list()
        
        def parse_html(html):
            """
            Parsing from html to text
            
            :param html: html link
        
            :return text: returning text as output
            """  
            page = requests.get(html)
            elem = BeautifulSoup(page.content, features="html.parser")
            results = elem.find('script',string=lambda text: 'location' in text.lower())
            if results is None:
                text = ''
                return text
            text = ''
            #print(results)
            for e in results.descendants:
                if isinstance(e, str):
                    text += e.strip()
                elif e.name in ['br',  'p', 'h1', 'h2', 'h3', 'h4','tr', 'th']:
                    text += '\n'
                elif e.name == 'li':
                    text += '\n- '
            return text
        
        
        results_text = parse_html(post_list)
        if not results_text:
            pass 
            
        start = [m.start() for m in re.finditer('address', results_text)]
        end = start[1:]
        end.append(results_text.find('sidebar')-1)
        
        #Separate addresses
        items= list()
        for i in range(0,len(start)):
            substring = results_text[start[i]-2:end[i]-3]
            res = json.loads(substring)
            items.append(res)
        
        if not items:
            return list()
        else:
            for i in range(len(items)):
                for trans_no in range(len(items[i].get("transactions"))):
                #    print(items)
                    address.append(items[i].get("address")) 
                    propertyType.append(items[i].get("propertyType"))
                    bedrooms.append(items[i].get("bedrooms"))
                    bathrooms.append(items[i].get("bathrooms"))
                    
                    transaction = items[i].get("transactions")[trans_no]
                    transactions_price.append(transaction.get("displayPrice"))
                    transactions_date.append(transaction.get("dateSold"))
                    transactions_tenure.append(transaction.get("tenure"))
                    
                    loc = items[i].get("location")
                    lat.append(loc.get("lat"))
                    lgt.append(loc.get("lng"))
                    detailUrl.append(items[i].get("detailUrl"))
                    
            data = {'address': address, 
                    'propertyType': propertyType, 
                    'bedrooms':bedrooms, 
                    'bathrooms':bathrooms,
                    'transactions_price':transactions_price,
                    'transactions_date':transactions_date,
                    'transactions_tenure':transactions_tenure,
                    'lat':lat,
                    'lgt':lgt,
                    'detailUrl':detailUrl}
            return data
    
    master = [get_data_postcode(i) for i in set(post_list_rightmove)]
    master_filtered = [x for x in master if x]
    if not master_filtered:
        data_year = []
        return data_year
        
    df = pd.DataFrame(master_filtered[0])
    #df = pd.DataFrame(reduce(lambda a, b: dict(a, **b), master_filtered))
    
    df['transactions_date_dt'] = df['transactions_date'].apply(lambda x: datetime.strptime(x, '%d %b %Y'))
    data_year = df[df['transactions_date_dt']>=str(user_input_year)+'-01-01 00:00:00']
    
    data_year['postcode'] = user_input
    
    return data_year

In [42]:
# Insert postcode data into database

conn = pymysql.connect(host='xxxx', 
                       user='xxxx', 
                       password='xxxx')

cursor = conn.cursor()
cursor.execute("select database();")
cursor.connection.commit()
sql = '''use postcodes'''
cursor.execute(sql)
    
for postcd in AL['Postcode']:

    sql = "select * from properties where postcode='{}'".format(postcd)
    cursor.execute(sql)
    
    existing = pd.DataFrame.from_records(cursor.fetchall(), columns=['address', 'propertyType', 'bedrooms', 'bathrooms',
           'transactions_price', 'transactions_date', 'transactions_tenure', 'lat',
           'lgt', 'detailUrl', 'transactions_date_dt', 'postcode'])
    
    df = get_data(postcd, 2010)
    
    if len(df)==0:
        print("Postcode: {} is empty".format(postcd))
        continue
        
    print("Scraped postcode:{}".format(postcd))
    
    df = df.where(pd.notnull(df), None)
    
    #Only include records that are not already in the database
    df_output = df_drop_df(existing, df, l_columns = ['address', 'transactions_date'])
    
    if not df_output.empty:
        for i,row in df_output.iterrows():
            sql = "INSERT INTO postcodes.properties VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
            cursor.execute(sql, tuple(row))
            conn.commit()
    print("Records inserted")

Postcode: al55uq is empty
Postcode: al55ur is empty
Postcode: al55uw is empty
Postcode: al55wa is empty
Postcode: al55wb is empty
Postcode: al55wd is empty
Postcode: al55wz is empty
Postcode: al55xj is empty
Postcode: al55xn is empty
Postcode: al55xq is empty
Postcode: al55xs is empty
Postcode: al55xz is empty
Postcode: al55yn is empty
Postcode: al55yz is empty
Postcode: al55zd is empty
Postcode: al55ze is empty
Postcode: al55zf is empty
Postcode: al55zg is empty
Postcode: al55zh is empty
Postcode: al55zj is empty
Postcode: al55zl is empty
Postcode: al55zn is empty
Postcode: al55zp is empty
Postcode: al55zq is empty
Postcode: al55zr is empty
Postcode: al55zs is empty
Postcode: al55zt is empty
Postcode: al55zu is empty
Postcode: al55zw is empty
Postcode: al55zx is empty
Postcode: al55zy is empty
Postcode: al55zz is empty
Postcode: al59aa is empty
Postcode: al59ab is empty
Postcode: al59ad is empty
Postcode: al59ae is empty
Postcode: al59af is empty
Postcode: al59ag is empty
Postcode: al

<ipython-input-15-78119cdc467b>:144: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_year['postcode'] = user_input


Scraped postcode:al60aa
Records inserted
Scraped postcode:al60ab
Records inserted
Postcode: al60ad is empty
Scraped postcode:al60ae
Records inserted
Postcode: al60af is empty
Scraped postcode:al60ag
Records inserted
Scraped postcode:al60ah
Records inserted
Scraped postcode:al60aj
Records inserted
Scraped postcode:al60al
Records inserted
Scraped postcode:al60an
Records inserted
Scraped postcode:al60ap
Records inserted
Scraped postcode:al60aq
Records inserted
Scraped postcode:al60ar
Records inserted
Scraped postcode:al60as
Records inserted
Scraped postcode:al60at
Records inserted
Scraped postcode:al60au
Records inserted
Scraped postcode:al60aw
Records inserted
Scraped postcode:al60ax
Records inserted
Postcode: al60ay is empty
Scraped postcode:al60az
Records inserted
Scraped postcode:al60ba
Records inserted
Scraped postcode:al60bb
Records inserted
Postcode: al60bd is empty
Scraped postcode:al60be
Records inserted
Postcode: al60bf is empty
Postcode: al60bg is empty
Scraped postcode:al60bh


Records inserted
Postcode: al60rt is empty
Scraped postcode:al60ru
Records inserted
Scraped postcode:al60rw
Records inserted
Scraped postcode:al60rx
Records inserted
Scraped postcode:al60ry
Records inserted
Scraped postcode:al60rz
Records inserted
Scraped postcode:al60sa
Records inserted
Scraped postcode:al60sb
Records inserted
Scraped postcode:al60sd
Records inserted
Scraped postcode:al60se
Records inserted
Scraped postcode:al60sf
Records inserted
Scraped postcode:al60sg
Records inserted
Scraped postcode:al60sh
Records inserted
Scraped postcode:al60sj
Records inserted
Scraped postcode:al60sl
Records inserted
Scraped postcode:al60sn
Records inserted
Scraped postcode:al60sp
Records inserted
Scraped postcode:al60sq
Records inserted
Scraped postcode:al60sr
Records inserted
Scraped postcode:al60ss
Records inserted
Postcode: al60st is empty
Postcode: al60su is empty
Scraped postcode:al60sw
Records inserted
Postcode: al60sx is empty
Postcode: al60sy is empty
Scraped postcode:al60sz
Records i

Records inserted
Postcode: al69ht is empty
Scraped postcode:al69hu
Records inserted
Postcode: al69hw is empty
Scraped postcode:al69hx
Records inserted
Scraped postcode:al69hy
Records inserted
Scraped postcode:al69hz
Records inserted
Scraped postcode:al69ja
Records inserted
Postcode: al69jb is empty
Scraped postcode:al69jd
Records inserted
Scraped postcode:al69je
Records inserted
Postcode: al69jf is empty
Scraped postcode:al69jg
Records inserted
Scraped postcode:al69jh
Records inserted
Scraped postcode:al69jj
Records inserted
Postcode: al69jl is empty
Scraped postcode:al69jn
Records inserted
Scraped postcode:al69jp
Records inserted
Scraped postcode:al69jq
Records inserted
Scraped postcode:al69jr
Records inserted
Postcode: al69js is empty
Postcode: al69jt is empty
Scraped postcode:al69ju
Records inserted
Scraped postcode:al69jw
Records inserted
Postcode: al69jx is empty
Scraped postcode:al69jy
Records inserted
Scraped postcode:al69jz
Records inserted
Scraped postcode:al69la
Records inser

...

Postcode: al86ne is empty
Postcode: al86nf is empty
Postcode: al86ng is empty
Postcode: al86nh is empty
Postcode: al86nj is empty
Postcode: al86nl is empty
Postcode: al86nn is empty
Postcode: al86np is empty
Postcode: al86nq is empty
Postcode: al86nr is empty
Postcode: al86ns is empty
Postcode: al86nt is empty
Postcode: al86nu is empty
Postcode: al86nw is empty
Postcode: al86nx is empty
Postcode: al86ny is empty
Postcode: al86nz is empty
Scraped postcode:al86pa
Records inserted
Scraped postcode:al86pb
Records inserted
Scraped postcode:al86pd
Records inserted
Postcode: al86pe is empty
Postcode: al86pf is empty
Postcode: al86pg is empty
Postcode: al86ph is empty
Postcode: al86pj is empty
Scraped postcode:al86pl
Records inserted
Postcode: al86pn is empty
Postcode: al86pp is empty
Postcode: al86pq is empty
Postcode: al86pr is empty
Postcode: al86ps is empty
Postcode: al86pt is empty
Postcode: al86pu is empty
Postcode: al86pw is empty
Postcode: al86px is empty
Postcode: al86py is empty
Scra

Postcode: al87du is empty
Scraped postcode:al87dw
Records inserted
Postcode: al87dx is empty
Scraped postcode:al87dy
Records inserted
Scraped postcode:al87ea
Records inserted
Scraped postcode:al87eb
Records inserted
Scraped postcode:al87ed
Records inserted
Scraped postcode:al87ee
Records inserted
Scraped postcode:al87ef
Records inserted
Scraped postcode:al87eg
Records inserted
Scraped postcode:al87eh
Records inserted
Scraped postcode:al87ej
Records inserted
Postcode: al87el is empty
Scraped postcode:al87en
Records inserted
Scraped postcode:al87ep
Records inserted
Scraped postcode:al87eq
Records inserted
Postcode: al87er is empty
Postcode: al87es is empty
Scraped postcode:al87ew
Records inserted
Postcode: al87ft is empty
Scraped postcode:al87ha
Records inserted
Scraped postcode:al87hb
Records inserted
Scraped postcode:al87hd
Records inserted
Scraped postcode:al87he
Records inserted
Scraped postcode:al87hf
Records inserted
Scraped postcode:al87hg
Records inserted
Scraped postcode:al87hh


Postcode: al95as is empty
Scraped postcode:al95at
Records inserted
Postcode: al95au is empty
Scraped postcode:al95aw
Records inserted
Scraped postcode:al95ax
Records inserted
Scraped postcode:al95ay
Records inserted
Scraped postcode:al95az
Records inserted
Postcode: al95ba is empty
Postcode: al95bb is empty
Postcode: al95bd is empty
Scraped postcode:al95be
Records inserted
Postcode: al95bf is empty
Postcode: al95bg is empty
Postcode: al95bh is empty
Postcode: al95bj is empty
Postcode: al95bl is empty
Postcode: al95bn is empty
Postcode: al95bp is empty
Postcode: al95bq is empty
Postcode: al95bs is empty
Postcode: al95bt is empty
Postcode: al95bu is empty
Scraped postcode:al95bw
Records inserted
Scraped postcode:al95bx
Records inserted
Scraped postcode:al95by
Records inserted
Scraped postcode:al95bz
Records inserted
Postcode: al95da is empty
Scraped postcode:al95db
Records inserted
Postcode: al95dd is empty
Scraped postcode:al95de
Records inserted
Postcode: al95df is empty
Postcode: al95

Records inserted
Postcode: al96hr is empty
Scraped postcode:al96hs
Records inserted
Postcode: al96ht is empty
Scraped postcode:al96hu
Records inserted
Scraped postcode:al96hw
Records inserted
Postcode: al96hx is empty
Scraped postcode:al96hy
Records inserted
Postcode: al96hz is empty
Scraped postcode:al96ja
Records inserted
Scraped postcode:al96jb
Records inserted
Postcode: al96jd is empty
Scraped postcode:al96je
Records inserted
Scraped postcode:al96jf
Records inserted
Scraped postcode:al96jg
Records inserted
Postcode: al96jh is empty
Scraped postcode:al96jj
Records inserted
Postcode: al96jl is empty
Postcode: al96jn is empty
Postcode: al96jp is empty
Scraped postcode:al96jq
Records inserted
Scraped postcode:al96jr
Records inserted
Postcode: al96js is empty
Postcode: al96jt is empty
Postcode: al96jw is empty
Postcode: al96jx is empty
Scraped postcode:al96jy
Records inserted
Postcode: al96jz is empty
Scraped postcode:al96la
Records inserted
Scraped postcode:al96lb
Records inserted
Post

Records inserted
Scraped postcode:al97qr
Records inserted
Postcode: al97qs is empty
Scraped postcode:al97qt
Records inserted
Postcode: al97qu is empty
Postcode: al97qw is empty
Scraped postcode:al97qx
Records inserted
Scraped postcode:al97qy
Records inserted
Scraped postcode:al97qz
Records inserted
Postcode: al97ra is empty
Postcode: al97rb is empty
Scraped postcode:al97rl
Records inserted
Scraped postcode:al97rn
Records inserted
Scraped postcode:al97rp
Records inserted
Scraped postcode:al97rr
Records inserted
Scraped postcode:al97rs
Records inserted
Scraped postcode:al97rt
Records inserted
Scraped postcode:al97ru
Records inserted
Scraped postcode:al97rw
Records inserted
Scraped postcode:al97rx
Records inserted
Scraped postcode:al97ry
Records inserted
Scraped postcode:al97rz
Records inserted
Scraped postcode:al97sa
Records inserted
Scraped postcode:al97sb
Records inserted
Postcode: al97sd is empty
Postcode: al97se is empty
Scraped postcode:al97sl
Records inserted
Postcode: al97sn is em

In [ ]:
# Check

conn = pymysql.connect(host='xxxx', 
                       user='xxxx', 
                       password='xxxx')

cursor = conn.cursor()
cursor.execute("select database();")
cursor.connection.commit()
sql = '''use postcodes'''
cursor.execute(sql)

sql = "select * from properties"
cursor.execute(sql)

existing = pd.DataFrame.from_records(cursor.fetchall(), columns=['address', 'propertyType', 'bedrooms', 'bathrooms',
       'transactions_price', 'transactions_date', 'transactions_tenure', 'lat',
       'lgt', 'detailUrl', 'transactions_date_dt', 'postcode'])